In [3]:
import numpy as np
import pandas as pd
import random
import math

# Load data
mat = np.load("Amean_Alpha.npy") 
f = np.load("freq_Alpha.npy") 
counts_total = np.load("totcounts_Alpha.npy")

print("Shape of mat: ", np.shape(mat))
print("Shape of f: ", np.shape(f))
print("Shape of counts_total:", np.shape(counts_total))

N = counts_total[:,13]*100 #Selecting total sequences starting at last time step (I included the x100 to suppress noise)
x = f[:,0,13] #Selecting allele 0 at last time step


time_steps = []

# Compute the neutral dynamics and store in time_steps
for s in range(220):
    x_N = []

    #adding noise to x by sampling from binomial distrubution
    for i in range(len(x)):
        x_i = np.random.binomial(N[i], x[i], size=None) / N[i] #For each deme i
        x_N.append(x_i) #assembling the x vector with noise modification

    x_N = np.array(x_N)
    
    x = mat.dot(x_N) #computing the next time step
    time_steps.append(x) #storing

# Generate index values as a separate column
index_column = list(range(len(time_steps)))

#time_steps is a list of lists, where each inner list has 9 elements (for 9 regions)
column_names = [f"Region_{i}" for i in range(9)]

# Convert results to a DataFrame
time_steps_df = pd.DataFrame(time_steps, columns=column_names)

# Set a synthetic datetime index with weekly frequency starting from a specific date (This is necessary for the transformer input)
date_index = pd.date_range(start='2023-01-01', periods=len(time_steps_df), freq='W')

# Insert the date index as the first column
time_steps_df.insert(0, "Date", date_index)

# Save the DataFrame to CSV without the default DataFrame index
time_steps_df.to_csv("unofficial_data.csv", index=False)


# Print final values and shapes for reference
print("Final f_vec:", x)
print("f[:, 0, 13]:", f[:, 0, 13])


Shape of mat:  (9, 9)
Shape of f:  (9, 77, 14)
Shape of counts_total: (9, 14)
Final f_vec: [0.06087155 0.06530968 0.06255483 0.06555346 0.06498756 0.06662184
 0.06580113 0.06874638 0.07331006]
f[:, 0, 13]: [0.01212121 0.09195402 0.07159763 0.05546218 0.14159292 0.02234637
 0.04016913 0.05235602 0.        ]
